# Maximum entropy modeling

We consider a distribution $P(\boldsymbol \sigma)$, where $\boldsymbol \sigma$ is an N-dimensional state vector. We search for the distribution which maximizes the entropy subject to some constraints on the expectation value of a (smallish) number of observables:

$$\langle \sum_{\boldsymbol \sigma} P(\boldsymbol \sigma) f_\mu(\boldsymbol \sigma)\rangle = f_\mu^{emp}$$

Using the method of Lagrange multipliers we can show that the distributions take the form:

$$P(\boldsymbol \sigma) = \frac{1}{Z} \exp\left[ -\sum_\mu \lambda_\mu f_\mu(\boldsymbol \sigma) \right]$$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from lib import *
%matplotlib inline

In [2]:
class MaxEntModel:
    def __init__(self, N, q, constraints, prng=None):
        """
        N: number of spins
        q: number of possible spin states
        constraints: list of constraints
        """
        self.N = N
        self.q = q
        self.constraints = constraints
        self.lambdas = np.zeros_like(constraints)
        if prng is None:
            self.prng = np.random
        else:
            self.prng = prng
    def energy(self, sigma):
        return np.sum(self.lambdas * np.array([c(sigma) for c in self.constraints]))
    def sample(self, n):
        'n: number of samples'
        def jump(x):
            return self.prng.randint(self.q, size=self.N)
        x0 = jump(np.zeros(self.N))
        return mcmcsampler(x0, self.energy, jump, n)

In [3]:
def gen_field_constraint(index):
    def constraint(x):
        return x[index]
    return constraint

In [4]:
m = MaxEntModel(5, 2, [gen_field_constraint(i) for i in range(5)])
m.lambdas = np.array([0.1, 0.2, 0.3, -0.1, -0.3])
m.energy([0, 1, 1, 0, 0])

0.5

In [5]:
m.sample(10)

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 1, 1, 0, 1],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 1]])